In [3]:
# Import the libraries
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

In [4]:
# install yfinance package if needed
%pip install yfinance
%pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import pandas as pd
import datetime
from datetime import date

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import yfinance as yf

import warnings
warnings.filterwarnings('ignore')

print('✔️ Libraries Loaded!')



✔️ Libraries Loaded!


# custom function

In [7]:
def get_closed_dates(df):
    """Return a list containing all dates on which the stock market was closed."""
    # Create a dataframe that contains all dates from the start until today.
    timeline = pd.date_range(start=df['Date'].iloc[0], end=df['Date'].iloc[-1])

    # Create a list of the dates existing in the dataframe.
    df_dates = [day.strftime('%Y-%m-%d') for day in pd.to_datetime(df['Date'])]

    # Finally, determine which dates from the 'timeline' do not exist in our dataframe.
    closed_dates = [
        day for day in timeline.strftime('%Y-%m-%d').tolist()
        if not day in df_dates
    ]

    return closed_dates


print('✔️ Function defined!')

##############################################################################################################################


def get_MACD(df, column='Adj Close', ema1 = 12, ema2 = 26):
    """Return a new DataFrame with the MACD and related information (signal line and histogram)."""
    df['EMA-12'] = df[column].ewm(span=12, adjust=False).mean()
    df['EMA-26'] = df[column].ewm(span=26, adjust=False).mean()

    # MACD Indicator = 12-Period EMA − 26-Period EMA.
    df['MACD'] = df['EMA-12'] - df['EMA-26']

    # Signal line = 9-day EMA of the MACD line.
    df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

    # Histogram = MACD - Indicator.
    df['Histogram'] = df['MACD'] - df['Signal']

    return df


print('✔️ Function defined!')

##############################################################################################################################


def get_MA_BB_VOL(df, column='Adj Close', BB_WINDOW_SIZE = 20, VOL_WINDOW_SIZE = 5):
    """Return a new DataFrame with the MACD and related information (signal line and histogram)."""

    df['SMA_5']  = df[column].rolling(window=5).mean()
    df['SMA_10'] = df[column].rolling(window=10).mean()
    df['SMA_20'] = df[column].rolling(window=20).mean()
    df['SMA_50'] = df[column].rolling(window=50).mean()

    std = df[column].rolling(window=BB_WINDOW_SIZE).std()

    df['upper_bb'] = df.SMA_20 + std*2
    df['lower_bb'] = df.SMA_20 - std*2

    df['ma_vol'] = df['Volume'].rolling(window=VOL_WINDOW_SIZE).mean()
    df['max_5d'] = df[column].rolling(window=VOL_WINDOW_SIZE).max()
    df['min_5d'] = df[column].rolling(window=VOL_WINDOW_SIZE).min()

    return df

# optmization
### include history maximum drawdown 
### define a string trading environment & only perform BB strategy in this period


print('✔️ Function defined!')

##############################################################################################################################


def get_RSI(df, column='Adj Close', time_window=14):
    """Return the RSI indicator for the specified time window."""
    diff = df[column].diff(1)

    # This preservers dimensions off diff values.
    up_chg = 0 * diff
    down_chg = 0 * diff

    # Up change is equal to the positive difference, otherwise equal to zero.
    up_chg[diff > 0] = diff[diff > 0]

    # Down change is equal to negative deifference, otherwise equal to zero.
    down_chg[diff < 0] = diff[diff < 0]

    # We set com = time_window-1 so we get decay alpha=1/time_window.
    up_chg_avg = up_chg.ewm(com=time_window - 1,
                            min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window - 1,
                                min_periods=time_window).mean()

    RS = abs(up_chg_avg / down_chg_avg)
    df['RSI'] = 100 - 100 / (1 + RS)

    return df


print('✔️ Function defined!')

##############################################################################################################################


def get_trading_strategy(df, column='Adj Close'):
    """Return the Buy/Sell signal on the specified (price) column (Default = 'Adj Close')."""
    buy_list, sell_list = [], []
    flag = False

    for i in range(0, len(df)):
        if df['MACD'].iloc[i] > df['Signal'].iloc[i] and flag == False:
            buy_list.append(df[column].iloc[i])
            sell_list.append(np.nan)
            flag = True

        elif df['MACD'].iloc[i] < df['Signal'].iloc[i] and flag == True:
            buy_list.append(np.nan)
            sell_list.append(df[column].iloc[i])
            flag = False

        else:
            buy_list.append(np.nan)
            sell_list.append(np.nan)

    df['Buy'] = buy_list
    df['Sell'] = sell_list

    return df


print('✔️ Function defined!')

##############################################################################################################################
def get_trading_strategy_MACD(df, column='Adj Close'):
    """Return the Buy/Sell signal on the specified (price) column (Default = 'Adj Close')."""
    buy_list, sell_list = [], []
    flag = False

    for i in range(0, len(df)):
        if df['MACD'].iloc[i] > df['Signal'].iloc[i] and flag == False:
            buy_list.append(df[column].iloc[i])
            sell_list.append(np.nan)
            flag = True

        elif df['MACD'].iloc[i] < df['Signal'].iloc[i] and flag == True:
            buy_list.append(np.nan)
            sell_list.append(df[column].iloc[i])
            flag = False

        else:
            buy_list.append(np.nan)
            sell_list.append(np.nan)

    df['Buy_MACD'] = buy_list
    df['Sell_MACD'] = sell_list

    return df

print('✔️ Function defined!')

##############################################################################################################################
def get_trading_strategy_BB(df, column='Adj Close'):
    """Return the Buy/Sell signal on the specified (price) column (Default = 'Adj Close')."""
    buy_list, sell_list = [], []
    flag = False  # no stocks on hand

    for i in range(1, len(df)):

    # lower bound case
        if (df[column].iloc[i-1] > df['lower_bb'].iloc[i-1]) and (df[column].iloc[i] < df['lower_bb'].iloc[i]) and flag == False:   # need to optimize the trend and the SD for each stock
            if (df['lower_bb'].iloc[i] - df[column].iloc[i])/ df['lower_bb'].iloc[i] > 0.01:
                buy_list.append(df[column].iloc[i])
                sell_list.append(np.nan)
                flag = True
            else:
                buy_list.append(np.nan)
                sell_list.append(np.nan)               

        # elif (df[column].iloc[i-1] < df['lower_bb'].iloc[i-1]) and (df[column].iloc[i] > df['lower_bb'].iloc[i]) and flag == True:
        #     buy_list.append(np.nan)
        #     sell_list.append(df[column].iloc[i])
        #     flag = False

    # upper bound case
        # if (df[column].iloc[i-1] > df['upper_bb'].iloc[i-1]) and (df[column].iloc[i] < df['upper_bb'].iloc[i]) and flag == False:   # need to optimize the trend and the SD for each stock
        #     buy_list.append(df[column].iloc[i])
        #     sell_list.append(np.nan)
        #     flag = True

        elif (df[column].iloc[i-1] < df['upper_bb'].iloc[i-1]) and (df[column].iloc[i] > df['upper_bb'].iloc[i]) and flag == True:
            buy_list.append(np.nan)
            sell_list.append(df[column].iloc[i])
            flag = False

    # on hold

        else:
            buy_list.append(np.nan)
            sell_list.append(np.nan)
    
    df['Buy_BB'] = [np.nan] + buy_list
    df['Sell_BB'] = [np.nan] + sell_list

    return df


print('✔️ Function defined!')
##############################################################################################################################

def get_trading_strategy_VOL(df, column='Adj Close'):
    """Return the Buy/Sell signal on the specified (price) column (Default = 'Adj Close')."""
    buy_list, sell_list = [], []
    flag = False

    for i in range(1, len(df)):

        if (df[column].iloc[i] > df['max_5d'].iloc[i]) and (df['Volume'].iloc[i] > df['ma_vol'].iloc[i]) and flag == False:
            buy_list.append(df[column].iloc[i])
            sell_list.append(np.nan)
            flag = True

        elif (df[column].iloc[i] < df['min_5d'].iloc[i]) and (df['Volume'].iloc[i] > df['ma_vol'].iloc[i]) and flag == True:
            buy_list.append(np.nan)
            sell_list.append(df[column].iloc[i])
            flag = False

        else:
            buy_list.append(np.nan)
            sell_list.append(np.nan)

    df['Buy_VOL'] =  [np.nan] + buy_list
    df['Sell_VOL'] = [np.nan] + sell_list
    
    return df

print('✔️ Function defined!')


# if rising trend and big volume means its top / bottom. hv to define for string trade situation

##############################################################################################################################
def plot_candlestick_chart(fig, df, row, column=1, plot_EMAs=True, plot_BBs=True, plot_strategy=True):
    """Return a graph object figure containing a Candlestick chart in the specified row."""
    fig.add_trace(go.Candlestick(x=df['Date'],
                                 open=df['Open'],
                                 high=df['High'],
                                 low=df['Low'],
                                 close=df['Close'],
                                 name='Candlestick Chart'),
                  row=row,
                  col=column)

    # If the boolean argument plot_EMAs is True, then show the line plots for the two exponential moving averages.
    if (plot_EMAs == True):
        fig.add_trace(go.Scatter(x=df['Date'],
                                 y=df['EMA-12'],
                                 name='12-period EMA',
                                 line=dict(color='dodgerblue', width=2)),
                      row=row,
                      col=column)

        fig.add_trace(go.Scatter(x=df['Date'],
                                 y=df['EMA-26'],
                                 name='26-period EMA',
                                 line=dict(color='whitesmoke', width=2)),
                      row=row,
                      col=column)

    if (plot_BBs == True):
        fig.add_trace(go.Scatter(x=df['Date'],
                                 y=df['upper_bb'],
                                 name='Upper BB',
                                 line=dict(color='ORANGE', width=2)),
                      row=row,
                      col=column)

        fig.add_trace(go.Scatter(x=df['Date'],
                                 y=df['lower_bb'],
                                 name='Lower_bb',
                                 line=dict(color='ORANGE', width=2)),
                      row=row,
                      col=column)



    if (plot_strategy == True):

        fig.add_trace(go.Scatter(x=df['Date'],
                                    y=df['Buy_MACD'],
                                    name='Buy MACD',
                                    mode='markers',
                                    marker_symbol='triangle-up',
                                    marker=dict(size=9),
                                    line=dict(color='Lime')),
                        row=row,
                        col=column)

        fig.add_trace(go.Scatter(x=df['Date'],
                                    y=df['Sell_MACD'],
                                    name='Sell MACD',
                                    mode='markers',
                                    marker_symbol='triangle-down',
                                    marker=dict(size=9, color='Yellow')),
                        row=row,
                        col=column)

        fig.add_trace(go.Scatter(x=df['Date'],
                                    y=df['Buy_BB'],
                                    name='Buy BB',
                                    mode='markers',
                                    marker_symbol='triangle-up',
                                    marker=dict(size=9),
                                    line=dict(color='skyblue')),
                        row=row,
                        col=column)

        fig.add_trace(go.Scatter(x=df['Date'],
                                    y=df['Sell_BB'],
                                    name='Sell BB',
                                    mode='markers',
                                    marker_symbol='triangle-down',
                                    marker=dict(size=9, color='aqua')),
                        row=row,
                        col=column)

        fig.add_trace(go.Scatter(x=df['Date'],
                                    y=df['Buy_VOL'],
                                    name='Buy VOL',
                                    mode='markers',
                                    marker_symbol='triangle-up',
                                    marker=dict(size=9),
                                    line=dict(color='blue')),
                        row=row,
                        col=column)

        fig.add_trace(go.Scatter(x=df['Date'],
                                    y=df['Sell_VOL'],
                                    name='Sell VOL',
                                    mode='markers',
                                    marker_symbol='triangle-down',
                                    marker=dict(size=9, color='turquoise')),
                        row=row,
                        col=column)
    fig.update_xaxes(rangeslider={'visible': False})
    fig.update_yaxes(title_text='Price ($)', row=row, col=column)

    return fig


print('✔️ Function defined!')










##############################################################################################################################

def plot_MACD(fig, df, row, column=1):
    """Return a graph object figure containing the MACD indicator, the signal line, and a histogram in the specified row."""
    df['Hist-Color'] = np.where(df['Histogram'] < 0, 'red', 'green')
    fig.add_trace(go.Bar(x=df['Date'],
                         y=df['Histogram'],
                         name='Histogram',
                         marker_color=df['Hist-Color'],
                         showlegend=True),
                  row=row,
                  col=column)

    fig.add_trace(go.Scatter(x=df['Date'],
                             y=df['MACD'],
                             name='MACD',
                             line=dict(color='darkorange', width=2)),
                  row=row,
                  col=column)

    fig.add_trace(go.Scatter(x=df['Date'],
                             y=df['Signal'],
                             name='Signal',
                             line=dict(color='cyan', width=2)),
                  row=row,
                  col=column)

    fig.update_yaxes(title_text='MACD', row=row, col=column)

    return fig


print('✔️ Function defined!')

##############################################################################################################################

def plot_RSI(fig, df, row, column=1):
    """Return a graph object figure containing the RSI indicator in the specified row."""
    fig.add_trace(go.Scatter(x=df['Date'].iloc[30:],
                             y=df['RSI'].iloc[30:],
                             name='RSI',
                             line=dict(color='gold', width=2)),
                  row=row,
                  col=column)

    fig.update_yaxes(title_text='RSI', row=row, col=column)

    # Add one red horizontal line at 70% (overvalued) and green line at 30% (undervalued)
    for y_pos, color in zip([70, 30], ['Red', 'Green']):
        fig.add_shape(x0=df['Date'].iloc[1],
                      x1=df['Date'].iloc[-1],
                      y0=y_pos,
                      y1=y_pos,
                      type='line',
                      line=dict(color=color, width=2),
                      row=row,
                      col=column)

    # Add a text box for each line
    for y_pos, text, color in zip([64, 36], ['Overvalued', 'Undervalued'], ['Red', 'Green']):
        fig.add_annotation(x=df['Date'].iloc[int(df['Date'].shape[0] / 10)],
                           y=y_pos,
                           text=text,
                           font=dict(size=14, color=color),
                           bordercolor=color,
                           borderwidth=1,
                           borderpad=2,
                           bgcolor='lightsteelblue',
                           opacity=0.75,
                           showarrow=False,
                           row=row,
                           col=column)

    # Update the y-axis limits
    ymin = 25 if df['RSI'].iloc[30:].min() > 25 else df['RSI'].iloc[30:].min() - 5
    ymax = 75 if df['RSI'].iloc[30:].max() < 75 else df['RSI'].iloc[30:].max() + 5
    fig.update_yaxes(range=[ymin, ymax], row=row, col=column)

    return fig


print('✔️ Function defined!')

##############################################################################################################################


def plot_volume(fig, df, row, column=1):
    """Return a graph object figure containing the volume chart in the specified row."""
    fig.add_trace(go.Bar(x=df['Date'],
                         y=df['Volume'],
                         marker=dict(color='lightskyblue',
                                     line=dict(color='firebrick', width=0.1)),
                         showlegend=False,
                         name='Volume'),
                  row=row,
                  col=column)

    fig.update_xaxes(title_text='Date', row=row, col=column)
    fig.update_yaxes(title_text='Volume ($)', row=row, col=column)

    return fig


print('✔️ Function defined!')

##############################################################################################################################
# backtesting
def backtesting(df, strategy = 'MACD'):

    stra_buy = 'Buy_' + strategy
    stra_sell = 'Sell_' + strategy

    df = sk0991HK_hr
    df['get_in'] = pd.Series(dtype='int')
    df['get_out'] = pd.Series(dtype='int')
    df['return%'] = pd.Series(dtype='int')
    df['trad_days'] = pd.Series(dtype='int')
    df['max_draw_down'] = pd.Series(dtype='int')
  
    day=0
    max_draw_down = 0
    in_price = 0

    for i in range(len(df)):
        if pd.notnull(df[stra_buy].iloc[i]):
            in_price = df[stra_buy].iloc[i]
            day=0
            max_draw_down = 0
        
        elif pd.notnull(df[stra_sell].iloc[i]):
            out_price = df[stra_sell].iloc[i]
            df['trad_days'].iloc[i] = day 
            df['return%'].iloc[i] = ((df['Close'].iloc[i] - in_price)/in_price)*100
            df['max_draw_down'].iloc[i] = max_draw_down
            df['get_in'].iloc[i] = in_price
            df['get_out'].iloc[i] = out_price

        else:
            day+=1
            max_draw_down = max(((df['Close'].iloc[i] - in_price)/in_price)*100, max_draw_down)

    return df
    
print('✔️ Function defined!')

##############################################################################################################################





##############################################################################################################################




##############################################################################################################################


✔️ Function defined!
✔️ Function defined!
✔️ Function defined!
✔️ Function defined!
✔️ Function defined!
✔️ Function defined!
✔️ Function defined!
✔️ Function defined!
✔️ Function defined!
✔️ Function defined!
✔️ Function defined!
✔️ Function defined!


In [8]:
watchlist = [#'^IXIC',  '^HSI' ,        # index
'AAPL' , 'MSFT' ,               # US
'3800.HK','1211.HK' ,'0388.HK' ,                     # HK common
'0991.HK'                    # HK special
#'BTC-USD'                     # Crypto
]

watchlist_str = ' '.join(watchlist)
watchlist_str


'AAPL MSFT 3800.HK 1211.HK 0388.HK 0991.HK'

In [9]:
# documentaiton fo download (https://algotrading101.com/learn/yfinance-guide/)
data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers =  watchlist_str,                                  #"QQQ AAPL MSFT 3800.HK",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "6mo",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = True,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )



# df = yf.download('GOOG',
#             start='2010-01-01',
#             end='2018-12-31',
#             adjusted=True,
#             progress=False)

df = data.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=[0,1])
df['Ticker'] = df['Ticker'].str.replace('^','')
print(f'Downloaded {df.shape[0]} rows of data.')

sk_ls = df.Ticker.unique()
df.head()

[*********************100%***********************]  6 of 6 completed
Downloaded 760 rows of data.


,Date,Ticker,Close,High,Low,Open,Volume
0,2022-02-28,0388.HK,368.957214,381.073388,364.267062,381.073388,10477797.0
1,2022-02-28,0991.HK,1.540000,1.560000,1.490000,1.510000,34780000.0
2,2022-02-28,1211.HK,237.307190,238.106880,232.908916,237.307190,9266014.0
3,2022-02-28,3800.HK,2.860000,2.930000,2.740000,2.870000,182787487.0
4,2022-02-28,AAPL,164.649048,164.948195,161.966718,162.594926,95056600.0


In [10]:
# check if its crypto or not (currently crypto is 0 volume )  [To be deleted]
crypto_ls = ['BTC','ETH']
user_input_ls = []
for i in sk_ls:
    user_input = ('sk'+i+'_hr').replace('.','').replace('-','')

    # check if its crypto
    if any(ext in user_input for ext in crypto_ls):  
        globals()[user_input] = df[(df.Ticker == i)]
        print('yes', user_input)
    else:
        globals()[user_input] = df[(df.Ticker == i) & (df.Volume != 0)]
        print('no', user_input)

    user_input_ls.append(user_input)
print(user_input_ls)

no sk0388HK_hr
no sk0991HK_hr
no sk1211HK_hr
no sk3800HK_hr
no skAAPL_hr
no skMSFT_hr
['sk0388HK_hr', 'sk0991HK_hr', 'sk1211HK_hr', 'sk3800HK_hr', 'skAAPL_hr', 'skMSFT_hr']


In [11]:
for df in user_input_ls:
    print(df)
    df = eval(df)

    # MACD & RSI
    df = get_MACD(df,'Close')
    df = get_RSI(df,'Close')
    df = get_MA_BB_VOL(df,'Close')

    #df = get_trading_strategy(df,'Close')
    df = get_trading_strategy_MACD(df,'Close')
    df = get_trading_strategy_BB(df,'Close')
    df = get_trading_strategy_VOL(df,'Close')
    closed_dates_list = get_closed_dates(df)
    ticker = df['Ticker'].iloc[0]
    df.reset_index(drop=True, inplace=True)

sk0388HK_hr
sk0991HK_hr
sk1211HK_hr
sk3800HK_hr
skAAPL_hr
skMSFT_hr


In [12]:
test = skMSFT_hr
test[test['Close']<  test['lower_bb']]
test[['Close','lower_bb']].head()

,Close,lower_bb
0,297.464294,NaN
1,293.641357,NaN
2,298.858093,NaN
3,294.607056,NaN
4,288.573914,NaN


In [13]:
# MACD & RSI

sk3800HK_hr = get_MACD(sk3800HK_hr,'Close')
sk3800HK_hr = get_RSI(sk3800HK_hr,'Close')
sk3800HK_hr = get_MA_BB_VOL(sk3800HK_hr,'Close')
sk3800HK_hr.head()

,Date,Ticker,Close,High,Low,Open,Volume,EMA-12,EMA-26,MACD,...,lower_bb,ma_vol,max_5d,min_5d,Buy_MACD,Sell_MACD,Buy_BB,Sell_BB,Buy_VOL,Sell_VOL
0,2022-02-28,3800.HK,2.86,2.93,2.74,2.87,182787487.0,2.860000,2.860000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-03-01,3800.HK,2.90,3.04,2.85,2.96,178066545.0,2.866154,2.862963,0.003191,...,NaN,NaN,NaN,NaN,2.9,NaN,NaN,NaN,NaN,NaN
2,2022-03-02,3800.HK,2.88,2.95,2.85,2.88,97070800.0,2.868284,2.864225,0.004059,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-03-03,3800.HK,2.97,3.02,2.88,2.90,172343180.0,2.883933,2.872060,0.011873,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-03-04,3800.HK,2.92,2.99,2.80,2.90,273733232.0,2.889481,2.875611,0.013870,...,NaN,180800248.8,2.97,2.86,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
sk3800HK_hr = get_trading_strategy(sk3800HK_hr,'Close')

print('✔️ Final DataFrame is ready!')

✔️ Final DataFrame is ready!


# check the buying date inititated by MACD strategy

In [32]:
df[df['Buy_MACD'].notnull()]

,Date,Ticker,Close,High,Low,Open,Volume,EMA-12,EMA-26,MACD,...,min_5d,Buy_MACD,Sell_MACD,Buy_BB,Sell_BB,Buy_VOL,Sell_VOL,return%,trad_days,max_draw_down
22,2022-03-30,0991.HK,1.20,1.20,1.15,1.16,21196000.0,1.209916,1.289399,-0.079483,...,1.13,1.20,NaN,NaN,NaN,NaN,NaN,,,
50,2022-05-16,0991.HK,1.25,1.26,1.21,1.22,12036000.0,1.237017,1.242152,-0.005136,...,1.21,1.25,NaN,NaN,NaN,NaN,NaN,,,
80,2022-06-28,0991.HK,1.34,1.34,1.29,1.29,24458000.0,1.292725,1.292420,0.000306,...,1.28,1.34,NaN,NaN,NaN,NaN,NaN,,,
114,2022-08-16,0991.HK,1.46,1.49,1.44,1.44,13322000.0,1.402646,1.403341,-0.000695,...,1.32,1.46,NaN,NaN,NaN,NaN,NaN,,,


In [73]:
# backtesting
def backtesting(df, strategy = 'MACD'):

    stra_buy = 'Buy_' + strategy
    stra_sell = 'Sell_' + strategy

    df = sk0991HK_hr
    df['get_in'] = pd.Series(dtype='int')
    df['get_out'] = pd.Series(dtype='int')
    df['return%'] = pd.Series(dtype='int')
    df['trad_days'] = pd.Series(dtype='int')
    df['max_draw_down'] = pd.Series(dtype='int')
  

    day=0
    max_draw_down = 0
    in_price = 0

    for i in range(len(df)):
        if pd.notnull(df[stra_buy].iloc[i]):
            in_price = df[stra_buy].iloc[i]
            day=0
            max_draw_down = 0
        
        elif pd.notnull(df[stra_sell].iloc[i]):
            out_price = df[stra_sell].iloc[i]
            df['trad_days'].iloc[i] = day 
            df['return%'].iloc[i] = ((df['Close'].iloc[i] - in_price)/in_price)*100
            df['max_draw_down'].iloc[i] = max_draw_down
            df['get_in'].iloc[i] = in_price
            df['get_out'].iloc[i] = out_price

        else:
            day+=1
            max_draw_down = max(((df['Close'].iloc[i] - in_price)/in_price)*100, max_draw_down)

    return df

In [74]:
# check the trade records
test = backtesting(sk0991HK_hr)
test[test.get_in.notnull()]

,Date,Ticker,Close,High,Low,Open,Volume,EMA-12,EMA-26,MACD,...,Buy_BB,Sell_BB,Buy_VOL,Sell_VOL,return%,trad_days,max_draw_down,get_in,get_out,max_draw_down%
47,2022-05-11,0991.HK,1.23,1.27,1.22,1.21,18097639.0,1.240470,1.245106,-0.004636,...,NaN,NaN,NaN,NaN,2.499998,24.0,8.333325,1.20,1.23,NaN
67,2022-06-09,0991.HK,1.32,1.35,1.31,1.33,13954000.0,1.341482,1.315360,0.026122,...,NaN,NaN,NaN,NaN,5.600004,16.0,11.999998,1.25,1.32,NaN
98,2022-07-25,0991.HK,1.46,1.50,1.44,1.48,14376000.0,1.471929,1.425060,0.046869,...,NaN,NaN,NaN,NaN,8.955224,17.0,14.925367,1.34,1.46,NaN


In [67]:
# show the max draw down records
sk0991HK_hr[sk0991HK_hr.max_draw_down.notnull()]

,Date,Ticker,Close,High,Low,Open,Volume,EMA-12,EMA-26,MACD,...,Buy_BB,Sell_BB,Buy_VOL,Sell_VOL,return%,trad_days,max_draw_down,get_in,get_out,max_draw_down%
47,2022-05-11,0991.HK,1.23,1.27,1.22,1.21,18097639.0,1.240470,1.245106,-0.004636,...,NaN,NaN,NaN,NaN,2.499998,24.0,8.333325,1.20,1.23,NaN
67,2022-06-09,0991.HK,1.32,1.35,1.31,1.33,13954000.0,1.341482,1.315360,0.026122,...,NaN,NaN,NaN,NaN,5.600004,16.0,11.999998,1.25,1.32,NaN
98,2022-07-25,0991.HK,1.46,1.50,1.44,1.48,14376000.0,1.471929,1.425060,0.046869,...,NaN,NaN,NaN,NaN,8.955224,17.0,14.925367,1.34,1.46,NaN


In [312]:
# sk0388HK_hr
# sk0991HK_hr
# sk1211HK_hr
# sk3800HK_hr
# skAAPL_hr
# skMSFT_hr

df = sk0991HK_hr

In [313]:
########## Plot the four plots ##########
fig = make_subplots(rows=4,
                    cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.005,
                    row_width=[0.2, 0.3, 0.3, 0.8])

fig = plot_candlestick_chart(fig,
                             df,
                             row=1,
                             plot_EMAs=True,
                             plot_strategy=True)
fig = plot_MACD(fig, df, row=2)
fig = plot_RSI(fig, df, row=3)
fig = plot_volume(fig, df, row=4)

########## Customise the figure ##########
# Update xaxis properties
fig.update_xaxes(rangebreaks=[dict(values=closed_dates_list)],
                 range=[df['Date'].iloc[0] - datetime.timedelta(days=3), df['Date'].iloc[-1] + datetime.timedelta(days=3)])

# Update basic layout properties (width&height, background color, title, etc.)
fig.update_layout(width=800,
                  height=800,
                  plot_bgcolor='#0E1117',
                  paper_bgcolor='#0E1117',
                  title={
                      'text': '{} - Stock Dashboard'.format(ticker),
                      'y': 0.98
                  },
                  hovermode='x unified',
                  legend=dict(orientation='h',
                              xanchor='left',
                              x=0.05,
                              yanchor='bottom',
                              y=1.003))

# Customize axis parameters
axis_lw, axis_color = 2, 'white'
fig.update_layout(xaxis1=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  yaxis1=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  font=dict(color=axis_color))

fig.update_layout(xaxis2=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  yaxis2=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  font=dict(color=axis_color))

fig.update_layout(xaxis3=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  yaxis3=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  font=dict(color=axis_color))

fig.update_layout(xaxis4=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  yaxis4=dict(linewidth=axis_lw,
                              linecolor=axis_color,
                              mirror=True,
                              showgrid=False),
                  font=dict(color=axis_color))

fig.show()